In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import flopy

# Groundwater Modeling and Python Programming

In this exercise, we will use MODPATH to simulate advective transport with the Freyberg flow model. For this exercise, we will use the flow model created in `03_Intersection-Capabilities.ipynb`. If you did not finish `03_Intersection-Capabilities.ipynb`, open `03_Intersection-Capabilities-completed.ipynb` and run all cells.

The location of the contamination patch and the nodes that the define bounding cells of the patch are calculated below.

In [ ]:
# patch upper left and lower right
xmin, xmax = 250. * 1, 250. * 3
ymin, ymax = (40 - 14) * 250., (40 - 11) * 250. 
csx, csy = [xmin, xmin, xmax, xmax, xmin], [ymin, ymax, ymax, ymin, ymin]
(xmin, ymax), (xmax, ymin)

In [ ]:
# patch edge node coordinates
xmin_node, xmax_node = xmin + 125, xmax - 125
ymin_node, ymax_node = ymin + 125, ymax - 125
(xmin_node, ymax_node), (xmax_node, ymin_node)

--------------------------

### Define the workspace and model names.

In [ ]:
ws = 'work/ex03'
name = 'ex03'
name_mp = name + '_mp'
exe_name = 'mf6'

## Load the MODFLOW 6 Model

Load a simulation object using `flopy.mf6.MFSimulation().load()`.

In [ ]:
sim = flopy.mf6.MFSimulation().load(sim_name=name, exe_name=exe_name,
                                    sim_ws=ws)

Run the simulation.

In [ ]:
sim.run_simulation()

## Create and Run the MODPATH model

Load the groundwater flow model using `sim.get_model()`.

In [ ]:
gwf = sim.get_model('freyberg6')

Lets plot the model grid and the location of the contamination patch.

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, layer=0)

fig = plt.gcf()
fig.set_size_inches(3, 6)

mm.plot_bc('RIV')
mm.plot_bc('WEL')
mm.plot_ibound()

mm.plot_grid(lw=0.5, color='black')

ax = plt.gca()
ax.fill(csx, csy, color='#e534eb')

Find the row column location of the contamination patch using the groundwater model grid object `modelgrid.intersect()` method. The upper left and lower right node coordinates of the cells containing the contamination patch are (375, 7125) and (625, 6625).

In [ ]:
gwf.modelgrid.intersect(xmin_node, ymax_node), gwf.modelgrid.intersect(xmax_node, ymin_node) 

### Create the MODPATH particle data

In [ ]:
# create
locs = [0, 11, 1, 0, 13, 2]
sd = flopy.modpath.FaceDataType()
p = flopy.modpath.LRCParticleData(subdivisiondata=[sd],
                                  lrcregions=[[locs]])
# create forward particle group
fpth = name_mp + '.sloc'
pg = flopy.modpath.ParticleGroupLRCTemplate(particlegroupname='contaminant',
                                            particledata=p,
                                            filename=fpth)

### Create the MODPATH 7 files

In [ ]:
# create modpath files
mp = flopy.modpath.Modpath7(modelname=name_mp, flowmodel=gwf,
                            exe_name='mp7', model_ws=ws)
mpbas = flopy.modpath.Modpath7Bas(mp, porosity=0.1)
mpsim = flopy.modpath.Modpath7Sim(mp, simulationtype='pathline', 
                                  trackingdirection='forward',
                                  weaksinkoption='pass_through',
                                  weaksourceoption='pass_through',
                                  referencetime=0.,
                                  stoptimeoption='extend',
                                  particlegroups=pg)

### Write MODPATH 7 files and run the model

In [ ]:
# write modpath datasets
mp.write_input()

# run modpath
mp.run_model()

## Post-Process the MODFLOW and MODPATH Results


### Load MODFLOW and MODPATH results from the heads and pathline files

Load the MODFLOW heads

In [ ]:
hname = gwf.oc.head_filerecord.array[0][0]
hname

In [ ]:
fpth = os.path.join(ws, hname)
hobj = flopy.utils.HeadFile(fpth)
hds = hobj.get_data()

Load the pathline file

In [ ]:
ppth = os.path.join(ws, name_mp + '.mppth')
p = flopy.utils.PathlineFile(ppth)
p0 = p.get_alldata()

### Plot the heads and pathlines

In [ ]:
mm = flopy.plot.PlotMapView(model=gwf, layer=0)
mm.plot_array(hds, masked_values=[1e30])

fig = plt.gcf()
fig.set_size_inches(3, 6)

mm.plot_bc('RIV')
mm.plot_bc('WEL')
mm.plot_ibound()
mm.plot_pathline(p0, layer='all', color='blue', lw=0.75)

ax = plt.gca()
ax.fill(csx, csy, color='#e534eb', zorder=100, alpha=.75)